La tabla "tbl_flujo_reporte" debe resumir en cada "id_item" el contenido del campo NETO de la tabla "tbl_transacciones" según "ITEM_COD", "ANIO" y "MES"

Evaluar las siguientes opciones:
1. ir creando una columna en tabla "tbl_flujo_reporte" para resumir la información de cada mes/año
2. Modificar la estructura de la tabla "tbl_flujo_reporte" para que reciba un registro por cada acumulado de NETO por "ITEM_COD",   "ANIO" y "MES"

========================================================================================

1. **Crear una columna en la tabla "tbl_flujo_reporte" para resumir la información de cada mes/año**: 
* Esta opción implica agregar una nueva columna para cada mes/año en la tabla "tbl_flujo_reporte". Cada vez que se ingresen nuevos datos en la tabla "tbl_transacciones", tendrás que actualizar las columnas correspondientes en "tbl_flujo_reporte". 
* Esto podría ser eficiente si necesitas acceder rápidamente a los totales mensuales/año sin tener que realizar cálculos complejos en tiempo de consulta. 
* Sin embargo, podría volverse complicado si hay muchos meses/años diferentes o si la cantidad de datos es grande, ya que tendrías muchas columnas en la tabla "tbl_flujo_reporte".

2. **Modificar la estructura de la tabla "tbl_flujo_reporte" para que reciba un registro por cada acumulado de NETO por "ITEM_COD", "ANIO" y "MES"**: 
* En esta opción, en lugar de tener una columna para cada mes/año en "tbl_flujo_reporte", tendrías una fila por cada combinación de "ITEM_COD", "ANIO" y "MES", con una columna que almacena el total NETO correspondiente. 
* Esto sería más escalable y flexible, ya que no tendrías que modificar la estructura de la tabla cada vez que agregues un nuevo mes/año. 
* Además, permitiría consultas más flexibles y eficientes sobre los datos agregados.

En general, la opción 2 parece ser la más flexible y escalable, ya que te permite almacenar los totales NETO de manera estructurada y realizar consultas eficientes sobre ellos. Sin embargo, la elección final dependerá de tus necesidades específicas y de la complejidad de tu aplicación.

========================================================================================

Uno de los propósitos de la tabla "tbl_flujo_reporte" es que sirva como puente para tomar datos que alimentarán un reporte

* Si el propósito principal de la tabla "tbl_flujo_reporte" es alimentar un reporte, entonces la opción 2 parece ser la más adecuada. 
* Al tener una fila por cada combinación de "ITEM_COD", "ANIO" y "MES" con el total NETO correspondiente, puedes generar fácilmente los datos necesarios para el reporte utilizando consultas SQL simples.
* Además, al utilizar esta estructura, la tabla "tbl_flujo_reporte" actuará como un puente eficiente para obtener los datos agregados necesarios para el reporte, ya que los totales NETO estarán precalculados y disponibles en la tabla, lo que evitará la necesidad de realizar cálculos complejos en tiempo de consulta.

Por lo tanto, la opción 2 te proporcionará una estructura eficiente y escalable para alimentar el reporte, lo que facilitará el proceso de generación del mismo y garantizará un rendimiento óptimo, especialmente si estás trabajando con grandes volúmenes de datos.

========================================================================================

In [2]:
import pandas as pd

import mysql.connector
import configparser
from openpyxl import Workbook

import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font

In [3]:
# ============================================================================================================
# Crear un dataframe con los datos de la tabla tbl_flujo_reporte según rango definido en parámetros de entrada
# ============================================================================================================

def report_mes(anio_inicio, mes_inicio, anio_fin, mes_fin):

    # Leer las credenciales del archivo de configuración
    config = configparser.ConfigParser()
    config.read('config.ini')

    user = config['database']['user']
    password = config['database']['password']
    host = config['database']['host']
    database = config['database']['database']

    # Abrir la base de datos: establecer la conexión
    conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

    # ============================================================================================================

    # Consulta SQL para extraer datos de tbl_flujo_reporte
    query = f"""
        SELECT *
        FROM tbl_flujo_reporte
        WHERE (anio > {anio_inicio} OR (anio = {anio_inicio} AND mes >= {mes_inicio}))
        AND (anio < {anio_fin} OR (anio = {anio_fin} AND mes <= {mes_fin}))
    """

    # Ejecutar la consulta y cargar los datos en un DataFrame
    reporte_mes = pd.read_sql(query, conexion)

    # Cerrar la conexión a la base de datos
    conexion.close()

    return reporte_mes

# ============================================================================================================

# Crear un dataframe con los datos de la tabla tbl_flujo_reporte según rango definido en parámetros de entrada
# Ejemplo de uso:
reporte_mes = report_mes(2023, 1, 2023, 2)
# print(reporte_mes)

# Crea la columna 'periodo' concatenando las columnas 'anio', 'mes'
reporte_mes['periodo'] = pd.to_datetime(reporte_mes['anio'].astype(str) + '-' + reporte_mes['mes'].astype(str) )
reporte_mes.tail()

C:\Users\gusta\AppData\Local\Temp\ipykernel_12920\3356292400.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reporte_mes = pd.read_sql(query, conexion)


,id_item,id_item_cod,anio,mes,total_neto,periodo
33,1791,32,2023,2,-510629.00,2023-02-01
34,1792,35,2023,2,-1264000.00,2023-02-01
35,1793,38,2023,2,-776881.77,2023-02-01
36,1794,39,2023,2,-98000.00,2023-02-01
37,1795,40,2023,2,-292500.00,2023-02-01


In [4]:
reporte_mes

,id_item,id_item_cod,anio,mes,total_neto,periodo
0,1745,2,2023,1,-330000.00,2023-01-01
1,1746,3,2023,1,-393648.86,2023-01-01
2,1747,7,2023,1,-2220953.00,2023-01-01
3,1748,8,2023,1,-2350098.00,2023-01-01
4,1749,9,2023,1,39685569.00,2023-01-01
5,1750,11,2023,1,-3338960.00,2023-01-01
6,1751,12,2023,1,-115973.92,2023-01-01
7,1752,14,2023,1,2898.16,2023-01-01
8,1753,15,2023,1,-5632376.00,2023-01-01
9,1754,20,2023,1,-49278.91,2023-01-01


In [5]:
reporte_mes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_item      38 non-null     int64         
 1   id_item_cod  38 non-null     int64         
 2   anio         38 non-null     int64         
 3   mes          38 non-null     int64         
 4   total_neto   38 non-null     float64       
 5   periodo      38 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 1.9 KB


In [6]:
# ============================================================================================================
# Crea un dataframe resumen de reporte_mes con la suma de 'total_neto' por 'id_item_cod' y 'periodo'
# ============================================================================================================

# Crear el DataFrame reporte_final utilizando pivot_table
reporte_final = pd.pivot_table(reporte_mes, 
                               values='total_neto', 
                               index='id_item_cod', 
                               columns='periodo', 
                               aggfunc='sum')

# Conservar el formato datetime en los títulos de las columnas
reporte_final.columns = reporte_final.columns.strftime('%Y-%m')


reporte_final.reset_index(inplace = True)

# Mostrando el nuevo dataframe
reporte_final

periodo,id_item_cod,2023-01,2023-02
0,2,-330000.00,-330000.00
1,3,-393648.86,-456206.92
2,5,NaN,-37000.00
3,7,-2220953.00,-3558973.08
4,8,-2350098.00,-4631322.00
5,9,39685569.00,45085363.00
6,11,-3338960.00,-3826570.00
7,12,-115973.92,-112080.15
8,14,2898.16,2323.64
9,15,-5632376.00,-5281700.00


In [7]:
reporte_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  22 non-null     int64  
 1   2023-01      18 non-null     float64
 2   2023-02      20 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 656.0 bytes


In [8]:
# ============================================================================================================
# Crear un dataframe con los datos de la tabla tbl_saldo_mes según rango definido en parámetros de entrada
# ============================================================================================================

def saldo_mes(anio_inicio, mes_inicio, anio_fin, mes_fin):

    # Leer las credenciales del archivo de configuración
    config = configparser.ConfigParser()
    config.read('config.ini')

    user = config['database']['user']
    password = config['database']['password']
    host = config['database']['host']
    database = config['database']['database']

    # Abrir la base de datos: establecer la conexión
    conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

    # ============================================================================================================

    # Consulta SQL para extraer datos de tbl_saldo_mes
    query = f"""
        SELECT *
        FROM tbl_saldo_mes
        WHERE (anio > {anio_inicio} OR (anio = {anio_inicio} AND mes >= {mes_inicio}))
        AND (anio < {anio_fin} OR (anio = {anio_fin} AND mes <= {mes_fin}))
    """

    # Ejecutar la consulta y cargar los datos en un DataFrame
    saldos_mes = pd.read_sql(query, conexion)

    # Cerrar la conexión a la base de datos
    conexion.close()

    return saldos_mes

# ============================================================================================================

# Crear un dataframe con los datos de la tabla tbl_saldo_mes según rango definido en parámetros de entrada

# Ejemplo de uso:
inf_saldos = saldo_mes(2023, 1, 2023, 2)
# print(reporte_mes)

# Crea la columna 'periodo' concatenando las columnas 'anio', 'mes'
inf_saldos['periodo'] = pd.to_datetime(inf_saldos['anio'].astype(str) + '-' + inf_saldos['mes'].astype(str) )
inf_saldos

C:\Users\gusta\AppData\Local\Temp\ipykernel_12920\2277621016.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  saldos_mes = pd.read_sql(query, conexion)


,id_saldo_mes,fecha,mes,anio,saldo_inicial,neto_mes,saldo_final,periodo
0,30,2023-01-31,1,2023,39647234.51,-15151744.01,24495490.50,2023-01-01
1,31,2023-02-28,2,2023,24495490.50,-17049613.28,7445877.22,2023-02-01


In [9]:
inf_saldos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_saldo_mes   2 non-null      int64         
 1   fecha          2 non-null      object        
 2   mes            2 non-null      int64         
 3   anio           2 non-null      int64         
 4   saldo_inicial  2 non-null      float64       
 5   neto_mes       2 non-null      float64       
 6   saldo_final    2 non-null      float64       
 7   periodo        2 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 256.0+ bytes


In [10]:
# ============================================================================================================
# Crea un dataframe resumen de inf_saldos
# con la suma de 'total_neto' por 'id_item_cod' y 'periodo'
# ============================================================================================================

# Crear el DataFrame reporte_final utilizando pivot_table
saldos_def = pd.pivot_table(inf_saldos, 
                               values=['saldo_inicial', 'saldo_final'],
                               columns='periodo', 
                               aggfunc='sum')

# Conservar el formato datetime en los títulos de las columnas
saldos_def.columns = saldos_def.columns.strftime('%Y-%m')

saldos_def.reset_index(inplace = True)

# Mostrando el nuevo dataframe
saldos_def

periodo,index,2023-01,2023-02
0,saldo_final,24495490.50,7445877.22
1,saldo_inicial,39647234.51,24495490.50


In [11]:
saldos_def.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   index    2 non-null      object 
 1   2023-01  2 non-null      float64
 2   2023-02  2 non-null      float64
dtypes: float64(2), object(1)
memory usage: 176.0+ bytes


In [12]:
# Renombra la columna 'index' de la tabla saldos_final a 'id_item_cod'
saldos_def.rename(columns={'index':'id_item_cod'}, inplace=True)
saldos_def

periodo,id_item_cod,2023-01,2023-02
0,saldo_final,24495490.50,7445877.22
1,saldo_inicial,39647234.51,24495490.50


In [13]:
saldos_def.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  2 non-null      object 
 1   2023-01      2 non-null      float64
 2   2023-02      2 non-null      float64
dtypes: float64(2), object(1)
memory usage: 176.0+ bytes


In [14]:
# Cambia el contenido del registro en la columna 'id_item_cod' de la tabla saldos_final
saldos_def['id_item_cod'] = saldos_def['id_item_cod'].replace('saldo_inicial', '48')
saldos_def['id_item_cod'] = saldos_def['id_item_cod'].replace('saldo_final', '49')
saldos_def['id_item_cod'] = saldos_def['id_item_cod'].astype(int)

saldos_def

periodo,id_item_cod,2023-01,2023-02
0,49,24495490.50,7445877.22
1,48,39647234.51,24495490.50


In [15]:
saldos_def.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  2 non-null      int32  
 1   2023-01      2 non-null      float64
 2   2023-02      2 non-null      float64
dtypes: float64(2), int32(1)
memory usage: 168.0 bytes


In [16]:
print(type(reporte_final))

<class 'pandas.core.frame.DataFrame'>


In [17]:
# Concatenar los DataFrames
reporte_final = pd.concat([reporte_final, saldos_def], ignore_index=True)
reporte_final

periodo,id_item_cod,2023-01,2023-02
0,2,-330000.00,-330000.00
1,3,-393648.86,-456206.92
2,5,NaN,-37000.00
3,7,-2220953.00,-3558973.08
4,8,-2350098.00,-4631322.00
5,9,39685569.00,45085363.00
6,11,-3338960.00,-3826570.00
7,12,-115973.92,-112080.15
8,14,2898.16,2323.64
9,15,-5632376.00,-5281700.00


In [18]:
# =======================================================================================================================================
# En este punto se tiene un DataFrame con la información de saldos (iniciales y finales) y los registros resumidos de flujos de los meses
# =======================================================================================================================================

In [19]:
# ============================================================================================================
# Crear un dataframe con los datos de la tabla tbl_fcdir_items
# ============================================================================================================

def data_items():

    # Leer las credenciales del archivo de configuración
    config = configparser.ConfigParser()
    config.read('config.ini')

    user = config['database']['user']
    password = config['database']['password']
    host = config['database']['host']
    database = config['database']['database']

    # Abrir la base de datos: establecer la conexión
    conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

    # ============================================================================================================

    # Consulta SQL para extraer datos de tbl_fcdir_items
    query = f"""
        SELECT *
        FROM tbl_fcdir_items
    """

    # Ejecutar la consulta y cargar los datos en un DataFrame
    tabla_items = pd.read_sql(query, conexion)

    # Cerrar la conexión a la base de datos
    conexion.close()

    return tabla_items

# ============================================================================================================

# Crear un dataframe con los datos de la tabla tbl_flujo_reporte según rango definido en parámetros de entrada
# Ejemplo de uso:
tabla_items = data_items()
tabla_items.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id_item  45 non-null     int64 
 1   name     45 non-null     object
 2   id_grp   45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


C:\Users\gusta\AppData\Local\Temp\ipykernel_12920\4204913904.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabla_items = pd.read_sql(query, conexion)


In [20]:
tabla_items

,id_item,name,id_grp
0,1,Cash and equivalents,1
1,2,Accounting software,4
2,3,Bank comissions & related,4
3,4,Building cuote,4
4,5,Certificates,4
5,6,Chamber of Commerce,4
6,7,Cleaning service,4
7,8,Cleaning supplies,4
8,9,Collect from clients,4
9,10,Exchange rate,4


In [21]:
# ============================================================================================================
# Crear un dataframe con los datos de la tabla tbl_fcdir_grupos
# ============================================================================================================

def data_grupos():

    # Leer las credenciales del archivo de configuración
    config = configparser.ConfigParser()
    config.read('config.ini')

    user = config['database']['user']
    password = config['database']['password']
    host = config['database']['host']
    database = config['database']['database']

    # Abrir la base de datos: establecer la conexión
    conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

    # ============================================================================================================

    # Consulta SQL para extraer datos de tbl_fcdir_grupos
    query = f"""
        SELECT *
        FROM tbl_fcdir_grupos
    """

    # Ejecutar la consulta y cargar los datos en un DataFrame
    tabla_grupos = pd.read_sql(query, conexion)

    # Cerrar la conexión a la base de datos
    conexion.close()

    return tabla_grupos

# ============================================================================================================

# Crear un dataframe con los datos de la tabla tbl_flujo_reporte según rango definido en parámetros de entrada
# Ejemplo de uso:
tabla_grupos = data_grupos()
tabla_grupos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id_grp  6 non-null      int64 
 1   name    6 non-null      object
dtypes: int64(1), object(1)
memory usage: 224.0+ bytes


C:\Users\gusta\AppData\Local\Temp\ipykernel_12920\812418555.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabla_grupos = pd.read_sql(query, conexion)


In [22]:
tabla_grupos

,id_grp,name
0,1,CASH AND EQUIVALENTS
1,2,FINANCING
2,3,INVESTING
3,4,OPERATING
4,5,INITIAL BALANCE
5,6,FINAL BALANCE


In [23]:
# ============================================================================================================
# Actualiza 'reporte_final' con datos de 'tabla_items'
# ============================================================================================================

# Realizar la fusión basada en la columna 'id_item_cod' de la tabla 'reporte_mes' y 'id_item' de la tabla 'tabla_items'
reporte_final = pd.merge(reporte_final, tabla_items, left_on='id_item_cod', right_on='id_item', how='left')

# Eliminar la columna 'id_item' ya que se ha utilizado para hacer la fusión y ahora está duplicada
# reporte_final.drop('id_item_y', axis=1, inplace=True)

# Mostrar el DataFrame resultante con la información agregada
reporte_final.head()


,id_item_cod,2023-01,2023-02,id_item,name,id_grp
0,2,-330000.00,-330000.00,2,Accounting software,4
1,3,-393648.86,-456206.92,3,Bank comissions & related,4
2,5,NaN,-37000.00,5,Certificates,4
3,7,-2220953.00,-3558973.08,7,Cleaning service,4
4,8,-2350098.00,-4631322.00,8,Cleaning supplies,4


In [24]:
reporte_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  24 non-null     int64  
 1   2023-01      20 non-null     float64
 2   2023-02      22 non-null     float64
 3   id_item      24 non-null     int64  
 4   name         24 non-null     object 
 5   id_grp       24 non-null     int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 1.2+ KB


In [25]:
# ============================================================================================================
# Actualiza 'reporte_final' con datos de 'tabla_grupos'
# ============================================================================================================

# Realizar la fusión basada en la columna 'id_grp' de la tabla 'reporte_final' con la columna 'id_grp' de la tabla 'tabla_grupos'
reporte_final = pd.merge(reporte_final, tabla_grupos, left_on='id_grp', right_on='id_grp', how='left')
reporte_final

,id_item_cod,2023-01,2023-02,id_item,name_x,id_grp,name_y
0,2,-330000.00,-330000.00,2,Accounting software,4,OPERATING
1,3,-393648.86,-456206.92,3,Bank comissions & related,4,OPERATING
2,5,NaN,-37000.00,5,Certificates,4,OPERATING
3,7,-2220953.00,-3558973.08,7,Cleaning service,4,OPERATING
4,8,-2350098.00,-4631322.00,8,Cleaning supplies,4,OPERATING
5,9,39685569.00,45085363.00,9,Collect from clients,4,OPERATING
6,11,-3338960.00,-3826570.00,11,Facility: Electricity,4,OPERATING
7,12,-115973.92,-112080.15,12,Hosting & domains,4,OPERATING
8,14,2898.16,2323.64,14,Interests received,2,FINANCING
9,15,-5632376.00,-5281700.00,15,Internet,4,OPERATING


In [26]:
reporte_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  24 non-null     int64  
 1   2023-01      20 non-null     float64
 2   2023-02      22 non-null     float64
 3   id_item      24 non-null     int64  
 4   name_x       24 non-null     object 
 5   id_grp       24 non-null     int64  
 6   name_y       24 non-null     object 
dtypes: float64(2), int64(3), object(2)
memory usage: 1.4+ KB


In [27]:
# ============================================================================================================
# Depura columnas de 'reporte_final'
# ============================================================================================================
reporte_final.drop(columns=['id_item'], inplace=True)
reporte_final.rename(columns={'name_x': 'Concepto'}, inplace=True)
reporte_final.rename(columns={'name_y': 'Grupo'}, inplace=True)

reporte_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  24 non-null     int64  
 1   2023-01      20 non-null     float64
 2   2023-02      22 non-null     float64
 3   Concepto     24 non-null     object 
 4   id_grp       24 non-null     int64  
 5   Grupo        24 non-null     object 
dtypes: float64(2), int64(2), object(2)
memory usage: 1.2+ KB


In [28]:
# ============================================================================================================
# Cambia el orden de columnas en 'reporte_final'
# ============================================================================================================

# Extraer las columnas 'Concepto', 'id_grp' y 'Grupo'
concepto_col = reporte_final.pop('Concepto')
id_grp_col = reporte_final.pop('id_grp')
grupo_col = reporte_final.pop('Grupo')

# Insertar las columnas en la nueva ubicación
reporte_final.insert(1, 'Concepto', concepto_col)
reporte_final.insert(2, 'id_grp', id_grp_col)
reporte_final.insert(3, 'Grupo', grupo_col)

reporte_final


,id_item_cod,Concepto,id_grp,Grupo,2023-01,2023-02
0,2,Accounting software,4,OPERATING,-330000.00,-330000.00
1,3,Bank comissions & related,4,OPERATING,-393648.86,-456206.92
2,5,Certificates,4,OPERATING,NaN,-37000.00
3,7,Cleaning service,4,OPERATING,-2220953.00,-3558973.08
4,8,Cleaning supplies,4,OPERATING,-2350098.00,-4631322.00
5,9,Collect from clients,4,OPERATING,39685569.00,45085363.00
6,11,Facility: Electricity,4,OPERATING,-3338960.00,-3826570.00
7,12,Hosting & domains,4,OPERATING,-115973.92,-112080.15
8,14,Interests received,2,FINANCING,2898.16,2323.64
9,15,Internet,4,OPERATING,-5632376.00,-5281700.00


In [29]:
# ============================================================================================================
# Reemplaza valores NaN en 'reporte_final'
# ============================================================================================================

# Reemplazar los valores NaN en 'Concepto' por 0
reporte_final.fillna(0, inplace=True)
reporte_final

,id_item_cod,Concepto,id_grp,Grupo,2023-01,2023-02
0,2,Accounting software,4,OPERATING,-330000.00,-330000.00
1,3,Bank comissions & related,4,OPERATING,-393648.86,-456206.92
2,5,Certificates,4,OPERATING,0.00,-37000.00
3,7,Cleaning service,4,OPERATING,-2220953.00,-3558973.08
4,8,Cleaning supplies,4,OPERATING,-2350098.00,-4631322.00
5,9,Collect from clients,4,OPERATING,39685569.00,45085363.00
6,11,Facility: Electricity,4,OPERATING,-3338960.00,-3826570.00
7,12,Hosting & domains,4,OPERATING,-115973.92,-112080.15
8,14,Interests received,2,FINANCING,2898.16,2323.64
9,15,Internet,4,OPERATING,-5632376.00,-5281700.00


In [30]:
reporte_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  24 non-null     int64  
 1   Concepto     24 non-null     object 
 2   id_grp       24 non-null     int64  
 3   Grupo        24 non-null     object 
 4   2023-01      24 non-null     float64
 5   2023-02      24 non-null     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 1.2+ KB


In [31]:
# ============================================================================================================
# Exportar el DataFrame reporte_final a un archivo Excel
# ============================================================================================================

In [37]:
from openpyxl.utils.dataframe import dataframe_to_rows

# Ordenar DataFrame por 'id_grp'
reporte_final = reporte_final.sort_values(by='id_grp')


# Crear un nuevo archivo de Excel
wb = Workbook()
ws = wb.active

# Añadir las filas específicas
ws['B1'] = "Nombre de la empresa"       # Añandir datos en la celda B1
ws['B2'] = "Flujo de Caja Directo"      # Añandir datos en la celda B2
ws['B3'] = "Lapso informado"            # Añandir datos en la celda B3
ws['B4'] = "Informe en COP"             # Añandir datos en la celda B4
ws.append([])  # Espacio en blanco

# Encabezados de las columnas
ws.append(reporte_final.columns.tolist())

# ========= Código para 'id_grp' igual a 5 ==========

grp_5_data = reporte_final[reporte_final['id_grp'] == 5]

# Recorremos cada grupo único y sus valores
for grp_id, grp_data in grp_5_data.groupby('id_grp'):
    
    # Añadir las filas del DataFrame correspondientes a este grupo
    for row in dataframe_to_rows(grp_data, index=False, header=False):
        ws.append(row)

    ws.append([])  # Espacio en blanco entre grupos

# ========= Código para 'id_grp' diferente de 5 ==========

grp_not_5_6_data = reporte_final[(reporte_final['id_grp'] != 5) & (reporte_final['id_grp'] != 6)]

# Recorremos cada grupo único y sus valores
for grp_id, grp_data in grp_not_5_6_data.groupby('id_grp'):
    
    # Añadir las filas del DataFrame correspondientes a este grupo
    for row in dataframe_to_rows(grp_data, index=False, header=False):
        ws.append(row)

    # Calcular la suma de las columnas numéricas y agregarla al final de este grupo
    sum_row = [''] * len(reporte_final.columns)  # Crear una fila vacía con el mismo número de columnas

    # Iterar sobre las columnas numéricas (a partir de la sexta columna)
    for col in grp_data.columns[3:]:
        if pd.api.types.is_numeric_dtype(grp_data[col]):
            sum_row[reporte_final.columns.get_loc(col)] = grp_data[col].sum()  # Sumar los valores de esta columna
    
    ws.append(sum_row)  # Agregar la fila de suma al final del grupo
    ws.append([])  # Espacio en blanco entre grupos

# ========= Código para 'id_grp' igual a 6 ==========

grp_6_data = reporte_final[reporte_final['id_grp'] == 6]

# Recorremos cada grupo único y sus valores
for grp_id, grp_data in grp_6_data.groupby('id_grp'):
    
    # Añadir las filas del DataFrame correspondientes a este grupo
    for row in dataframe_to_rows(grp_data, index=False, header=False):
        ws.append(row)

    ws.append([])  # Espacio en blanco entre grupos

# ==================================================================================================

# Eliminar la primera columna de la hoja de cálculo
ws.delete_cols(1)


# Guardar el archivo de Excel
wb.save("informe.xlsx")
